In [221]:
import math
import numpy as np
import pandas as pd
from numpy.linalg import norm
from itertools import combinations
from numba import njit, prange, config, threading_layer, get_thread_id, set_num_threads

In [222]:
config.THREADING_LAYER = 'threadsafe'

config.THREADING_LAYER_PRIORITY = ['omp', 'tbb', 'workqueue']

In [223]:
config.NUMBA_DEFAULT_NUM_THREADS

12

In [224]:
set_num_threads(6)

In [225]:
# Algumas funções auxiliares

def vec(x,y,z)-> np.ndarray:
    '''Função para facilmente transformar coordenadas em um vetor numpy.'''
    return np.array([x,y,z],dtype=float)

def rotation_3d(vec, rot):
    '''Função que rotaciona um vetor 3d em torno dos eixos x,y,z, a partir
    de um vetor de rotação rot = [rot_x, rot_y, rot_z]. Nota: @ é um operador
    da biblioteca numpy que realiza a multiplicação de matrizes.'''

    if rot[0] != 0:
        Rx = np.array([[1,0,0],
                       [0,np.cos(rot[0]),-np.sin(rot[0])],
                       [0,np.sin(rot[0]),np.cos(rot[0])]])
        vec = Rx @ vec
    
    if rot[1] != 0:
        Ry = np.array([[np.cos(rot[1]),0,np.sin(rot[1])],
                       [0,1,0],
                       [-np.sin(rot[1]),0,np.cos(rot[1])]])
        vec = Ry @ vec

    if rot[2] != 0:
        Rz = np.array([[np.cos(rot[2]),-np.sin(rot[2]),0],
                       [np.sin(rot[2]),np.cos(rot[2]),0],
                       [0,0,1]])
        vec = Rz @ vec 
        
    return vec

def unpack(combs, prop: int):
    '''Função auxiliar que desempacota uma lista de combinações de pares de grãos em 
    uma única lista com a propriedade desejada. Necessário para paralelizar a função apply_contacts().'''
    
    unpacked_list_1 = []
    unpacked_list_2 = []

    for i in range(len(combs)):
        unpacked_list_1.append(combs[i][0][prop])
        unpacked_list_2.append(combs[i][1][prop])
    
    return np.array(unpacked_list_1), np.array(unpacked_list_2)

In [226]:
def generate_grain(pos: iter, radius: float, density: float, vel: iter, grain_list: list)-> list:
    '''Função que gera um grão a partir dos parâmetros de entrada.'''

    pos = np.array(pos, dtype=float)
    vel = np.array(vel, dtype=float)
    mass = density*np.pi*(radius**3)*(4/3)
    acc = vec(0,0,0)
    force = vec(0,0,0)
    index = len(grain_list)

    grain = [index, pos, vel, acc, force, radius, mass]

    return grain

In [227]:
#@njit(parallel=True)
def cube_walls_parallel(gr_pos_x, gr_pos_y, gr_pos_z,
                        gr_vel_x, gr_vel_y, gr_vel_z,
                        gr_radius_list, center, scale, K):
    '''Função auxiliar que realiza o loop principal da função cube_walls() de forma paralela.'''

    x,y,z = center

    for i in range(len(gr_radius_list)): # Loop paralelizado. Nota: prange não permite step diferente de 1. Por isso a necessidade de usar 3 arrays separados para as posições e velocidades.

            p = np.array([gr_pos_x[i], gr_pos_y[i], gr_pos_z[i]])

            vel = np.array([gr_vel_x[i], gr_vel_y[i], gr_vel_z[i]])

            radius = gr_radius_list[i]

            if p[0] > x + scale - radius:
                p[0] = x +  scale - radius
                vel[0] = -vel[0]*K

            if p[0] < x - scale + radius:
                p[0] = x - scale + radius
                vel[0] = -vel[0]*K
            
            if p[1] > y + scale - radius:
                p[1] = y + scale - radius
                vel[1] = -vel[1]*K
            
            if p[1] < y - scale + radius:
                p[1] = y - scale + radius
                vel[1] = -vel[1]*K

            if p[2] > z + scale - radius:
                p[2] = z + scale - radius
                vel[2] = -vel[2]*K
            
            if p[2] < z - scale + radius:
                p[2] = z - scale + radius
                vel[2] = -vel[2]*K

            gr_pos_x[i], gr_pos_y[i], gr_pos_z[i] = p
            gr_vel_x[i], gr_vel_y[i], gr_vel_z[i] = vel

    return gr_pos_x, gr_pos_y, gr_pos_z, gr_vel_x, gr_vel_y, gr_vel_z

def cube_walls(center: iter, scale: float, grain_list: iter, K: float, rot = np.array([0.,0.,0.])):
    '''Função que gerencia a geometria do cubo, e a força de contato nas 
    partículas que estão em contato com as paredes do cubo.'''

    vectors = np.array([[1.,0.,0.], [0.,1.,0.],[0.,0.,1.]])

    if not rot.all == 0:
        rot_vectors = [rotation_3d(x, rot) for x in vectors]
        vectors = rot_vectors.copy()
    
    n_vectors = [x/norm(x) for x in vectors]
    vectors = n_vectors.copy()

    t_matrix = np.array(vectors).T

    inv_matrix = np.linalg.inv(t_matrix)

    gr_pos_list = np.array([gr[1] for gr in grain_list])
    gr_pos_list = np.array([t_matrix @ gr_pos for gr_pos in gr_pos_list])  
    gr_pos_x, gr_pos_y, gr_pos_z = gr_pos_list[:,0],  gr_pos_list[:,1] , gr_pos_list[:,2]

    gr_vel_list = np.array([gr[2] for gr in grain_list])
    gr_vel_x, gr_vel_y, gr_vel_z = gr_vel_list[:,0],  gr_vel_list[:,1] , gr_vel_list[:,2]

    gr_radius_list = np.array([gr[5] for gr in grain_list])

    gr_pos_x, gr_pos_y, gr_pos_z, gr_vel_x, gr_vel_y, gr_vel_z = cube_walls_parallel(gr_pos_x, gr_pos_y, gr_pos_z, gr_vel_x, gr_vel_y, gr_vel_z, gr_radius_list, center, scale, K)

    for i in range(len(grain_list)):
        gr_pos_list[i] = np.array([gr_pos_x[i], gr_pos_y[i], gr_pos_z[i]])
        gr_vel_list[i] = np.array([gr_vel_x[i], gr_vel_y[i], gr_vel_z[i]])

    gr_pos_list = np.array([inv_matrix @ gr_pos for gr_pos in gr_pos_list])

    for i in range(len(grain_list)):
        grain_list[i][1] = gr_pos_list[i]
        grain_list[i][2] = gr_vel_list[i]

In [228]:

def contact(pos_x1, pos_x2,
            pos_y1, pos_y2,
            pos_z1, pos_z2,
            vel_x1, vel_x2,
            vel_y1, vel_y2,
            vel_z1, vel_z2,
            radius_1, radius_2):
    '''Função que gerencia as forças de contato entre dois grãos.'''

    ks = 1e3
    kd = 1e3
    kt = 0.5
    mu = 0.2

    # Força Normal

    pos1, pos2 = np.array([pos_x1, pos_y1, pos_z1]), np.array([pos_x2, pos_y2, pos_z2])
    vel1, vel2 = np.array([vel_x1, vel_y1, vel_z1]), np.array([vel_x2, vel_y2, vel_z2])
    force1, force2 = np.array([0.,0.,0.]) , np.array([0.,0.,0.])
    r1, r2 = radius_1, radius_2
    
    csi = max(0, r1 + r2 - norm(pos1 - pos2))

    N = pos1 - pos2

    Fs = ks*csi*N/norm(N)

    Fd = kd*N

    Fn = Fs + Fd

    force1 += Fn
    force2 -= Fn

    # Força Tangencial

    Vt = vel1 - vel2

    Vt_norm = norm(Vt)

    if Vt_norm == 0:
        Ft = np.array([0.,0.,0.])
    else:
        Ft = -min(mu*norm(Fn), kt*norm(Vt))*Vt/Vt_norm

    force1 += Ft
    force2 -= Ft

    force_x1, force_y1, force_z1 = force1
    force_x2, force_y2, force_z2 = force2

    return force_x1, force_x2, force_y1, force_y2, force_z1, force_z2

def apply_contacts_parallel(gr_pos_x1, gr_pos_y1, gr_pos_z1,
                            gr_pos_x2, gr_pos_y2, gr_pos_z2,
                            gr_vel_x1, gr_vel_y1, gr_vel_z1,
                            gr_vel_x2, gr_vel_y2, gr_vel_z2,
                            gr_radius_1, gr_radius_2,
                            gr_norm, gr_index_1, gr_index_2):
    
    '''Função auxiliar que realiza o loop principal da função apply_contacts() de forma paralela.'''

    gr_force_x1, gr_force_y1, gr_force_z1 = np.zeros(len(gr_pos_x1)), np.zeros(len(gr_pos_x1)), np.zeros(len(gr_pos_x1))
    gr_force_x2, gr_force_y2, gr_force_z2 = np.zeros(len(gr_pos_x1)), np.zeros(len(gr_pos_x1)), np.zeros(len(gr_pos_x1))

    for i in range(len(gr_norm)): 

        print(gr_norm[i], gr_index_1[i], gr_index_2[i])

        if gr_norm[i] < (gr_radius_1[i] + gr_radius_2[i]):
            
            gr_force_x1[i], gr_force_x2[i], gr_force_y1[i], gr_force_y2[i], gr_force_z1[i], gr_force_z2[i] = contact(gr_pos_x1[i], gr_pos_x2[i],
                                                                                                                     gr_pos_y1[i], gr_pos_y2[i], 
                                                                                                                     gr_pos_z1[i], gr_pos_z2[i],
                                                                                                                     gr_vel_x1[i], gr_vel_x2[i], 
                                                                                                                     gr_vel_y1[i], gr_vel_y2[i], 
                                                                                                                     gr_vel_z1[i], gr_vel_z2[i],
                                                                                                                     gr_radius_1[i], gr_radius_2[i])
        
    return gr_force_x1, gr_force_x2, gr_force_y1, gr_force_y2, gr_force_z1, gr_force_z2



def apply_contacts(grain_list):
    '''Função que verifica se houve contato entre cada um dos pares de grãos
    por meio do cálculo de suas distâncias e, caso haja, aplica a função de
    força de contato contact().'''

    combs = list(combinations(grain_list, 2))

    gr_pos_1, gr_pos_2 = unpack(combs, 1)
    gr_vel_1, gr_vel_2 = unpack(combs, 2)
    gr_force_1, gr_force_2 = unpack(combs, 4)
    gr_radius_1, gr_radius_2 = unpack(combs, 5)
    gr_index_1, gr_index_2 = unpack(combs, 0)   

    # Separando em coordenadas
    
    gr_pos_x1, gr_pos_y1, gr_pos_z1 = gr_pos_1[:,0],  gr_pos_1[:,1] , gr_pos_1[:,2]
    gr_pos_x2, gr_pos_y2, gr_pos_z2 = gr_pos_2[:,0],  gr_pos_2[:,1] , gr_pos_2[:,2]

    gr_vel_x1, gr_vel_y1, gr_vel_z1 = gr_vel_1[:,0],  gr_vel_1[:,1] , gr_vel_1[:,2]
    gr_vel_x2, gr_vel_y2, gr_vel_z2 = gr_vel_2[:,0],  gr_vel_2[:,1] , gr_vel_2[:,2]

    gr_norm = norm(gr_pos_2 - gr_pos_1, axis=1)

    # Aplicando contato

    gr_force_x1, gr_force_x2, gr_force_y1, gr_force_y2, gr_force_z1, gr_force_z2 = apply_contacts_parallel( gr_pos_x1, gr_pos_y1, gr_pos_z1,
                                                                                                            gr_pos_x2, gr_pos_y2, gr_pos_z2,
                                                                                                            gr_vel_x1, gr_vel_y1, gr_vel_z1,
                                                                                                            gr_vel_x2, gr_vel_y2, gr_vel_z2,
                                                                                                            gr_radius_1, gr_radius_2,
                                                                                                            gr_norm, gr_index_1, gr_index_2)
    
    # Reagrupando

    gr_force_1 = np.array([gr_force_x1, gr_force_y1, gr_force_z1]).T
    gr_force_2 = np.array([gr_force_x2, gr_force_y2, gr_force_z2]).T

    

    for i in range(len(grain_list)):
        index_1, index_2 = gr_index_1[i], gr_index_2[i]
        grain_list[index_1][4] += gr_force_1[i]
        grain_list[index_2][4] += gr_force_2[i]

In [229]:

def apply_gravity(grain_list, g):
    '''Função que gerencia a força gravitacional aplicada somente aos grãos.'''
    for gr in grain_list:
        gr[4]+= gr[6]*g

def reset_force(grain_list):
    '''Função que coloca as forças de todos os grãos em zero. A finalidade
    dessa função é simples: se as forças não forem todas colocadas em zero
    no início de cada iteração, forças como a gravidade iriam se acumular
    de uma iteração para a outra, algo não desejado.'''

    for gr in grain_list:
        gr[4] = np.array([0., 0., 0.])

def verlet(grain_list, dt):
    '''Função que aplica a integração de verlet para atualizar as posições
    dos grãos.'''

    for gr in grain_list:
        a = gr[4]/gr[6]
        gr[2] += (gr[3] + a) * (dt/2.)
        gr[1] += gr[2] * dt + 0.5*a*(dt**2.)
        gr[3]  = a


def time_loop(grain_list, dt, g, center, scale, rot, K):
    '''Função que gerencia o laço temporal, aplicando cada uma das funções
    previamente definidas em ordem para cada iteração.'''
    
    reset_force(grain_list)
    apply_gravity(grain_list, g)
    apply_contacts(grain_list)
    cube_walls(center, scale, grain_list, K, rot)
    verlet(grain_list, dt)

In [230]:
# Funções auxiliares de geração de grãos

def generate_grains_random(n: int, radius: float, density: float, sidelength: float):
    '''Função que gera uma lista de grãos com posições e velocidades aleatórias,
    respeitando a escala do cubo e o raio dos grãos.'''
    
    grain_list = []
    sidelength -= 1.5*radius
    for _ in range(n):
        gr = generate_grain(vec(np.random.uniform(-sidelength/2, sidelength/2),
                       np.random.uniform(-sidelength/2, sidelength/2), 
                       np.random.uniform(0, sidelength)),
                    radius,
                    density,
                    vec(np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1)),
                    grain_list)
        grain_list.append(gr)
    return grain_list

def generate_grains_uniform(cube_scale: iter, n_max: iter, radius: float, density: float, randvel = False):
    '''Função que gera os grãos uniformemente distribuídos em uma grade, respeitando a escala
    do cubo e o raio dos grãos. É possível escolher uma escala menor que a do cubo e um número
    arbitrário de grãos, desde que caibam dentro do cubo com espaço. É possível também escolher
    se as velocidades iniciais serão todas nulas ou aleatórias.'''

    grain_list = []

    reduced_scale = cube_scale - 2*radius

    n_max = [min(int(r/radius), n) for r, n in zip(reduced_scale, n_max)]

    xs, ys, zs = reduced_scale

    for x in np.linspace(-xs, xs, n_max[0]):
        for y in np.linspace(-ys, ys, n_max[1]):
            for z in np.linspace(-zs, zs, n_max[2]):
                gr = generate_grain(vec(x,y,z), radius, density, vec(0.,0.,0.), grain_list)
                grain_list.append(gr)
    
    if randvel:
        for gr in grain_list:
            gr[2] = vec(np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1))

    return grain_list

In [231]:
# Funções auxiliares de movimentação do cubo

def linear_shake(center: iter, axis: int, T, dt, amplitude=1, omega=1)->np.ndarray:
    '''Função que translada o cubo em um eixo, de forma a simular uma vibração de um lado para o outro.'''

    cube_pos = []

    center = np.array(center,dtype=float)

    total_angle = 2*np.pi * T

    n_elements = np.arange(0,T,dt).shape[0]

    angle_range = np.linspace(0, total_angle,n_elements)

    omega = omega/(2*np.pi)

    for theta in angle_range:
        center[axis] = amplitude*np.sin(omega*theta)
        cube_pos.append(center.copy())

    return cube_pos

def angular_shake(rot: iter, axis: int, T, dt, amplitude=1, omega=1)->np.ndarray:
    '''Função que rota o cubo em um eixo e depois volta, de forma a simular uma vibração com rotação.'''

    cube_rot = []

    rot = np.array(rot,dtype=float)

    amplitude = amplitude * (np.pi/2) # Amplitude em radianos. Logo, amplitude = 1 -> rotação de 90°
    
    n_elements = np.arange(0,T,dt).shape[0]

    total_angle = 2*np.pi * T

    angle_range = np.linspace(0, total_angle, n_elements)

    omega = omega/(2*np.pi)

    for theta in angle_range:
        rot[axis] = amplitude*np.sin(omega*theta)
        cube_rot.append(rot.copy())

    return cube_rot

In [232]:
# Parâmetros

T = 2.5

dt = 0.001

K = .001

g = np.array([0.,0.,-9.81])

# Geometria do cubo

center = np.array((0.,0.,0.))

scale = 1

rot = np.array((0.,0.,0.))

time_steps = np.arange(0,T,dt)

# Aqui, no caso, o cubo não se move. Nota, ainda não foi implementada rotação do cubo.

# Grãos

cube_scale = np.array((1.,1.,1.))

n_max = (5,5,5)

radius = 0.05

density = 1600

randvel = False

grain_list = generate_grains_uniform(cube_scale, n_max, radius, density, randvel=randvel)

cube_pos = linear_shake(center = center,
                 axis = 2,
                 T = T,
                 dt =  dt,
                 omega=0,
                 amplitude=.5)

cube_rot = angular_shake(rot = rot,
                         axis = 1,
                         T = T,
                         dt = dt,
                         omega=0,
                         amplitude=.5)

In [233]:
# Execução da simulação


pos_list = [] # Lista de posições dos grãos

for i in prange(len(time_steps)):

    time_loop(grain_list, dt, g, center, scale, rot, K)
    
    temp_list = np.array([], float)
    
    for j in prange(len(grain_list)):
        gr = grain_list[j]
        for pos in np.array(gr[1]):
            temp_list = np.append(temp_list, pos)

    center = cube_pos[i]
    rot = cube_rot[i]

    pos_list.append(temp_list.copy())

TypeError: apply_contacts_parallel() missing 2 required positional arguments: 'gr_index_1' and 'gr_index_2'

In [ ]:
# Criando arquvios .csv completos para as posições dos grãos e do cubo

df = pd.DataFrame(pos_list)

df.to_csv('pos.csv')

pos_df = pd.DataFrame(cube_pos)

rot_df = pd.DataFrame(cube_rot)

cube_df = pd.merge(pos_df, rot_df, left_index=True, right_index=True)

cube_array = np.array(cube_df)

cube_df.to_csv('cube.csv')

In [ ]:
'''Precisamos cortar alguns frames para poder exportar ao Blender. O motivo para isso é que as animações
precisam ser renderizadas em no máximo 60 fps, mais do que isso é desperdício de poder computacional.
Então, cortamos a lista de quadros de forma a pegar um quadro a cada 60 quadros, ou seja, 60 quadros
por segundo. Essas listas serão carregadas no blender.'''

persec = 1/dt

fps = 60

frame_step = int(round(persec/fps,0))

cut_frames = pos_list[::frame_step]

df_frames = pd.DataFrame(cut_frames)

df_frames.to_csv('cut_frames.csv')

cube_cut_frames = cube_array[::frame_step]

cube_df_frames = pd.DataFrame(cube_cut_frames)

cube_df_frames.to_csv('cube_cut_frames.csv')